In [1]:
import numpy as np 
import pandas as pd 

In [3]:
import os
data_dir = '/home/ubuntu/notebooks/data/expedia'

In [5]:
train = pd.read_csv(os.path.join(data_dir, 'train.csv'),
                    dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32},
                    usecols=['srch_destination_id','is_booking','hotel_cluster'],
                    chunksize=1000000)
aggs = []
print('-'*38)
for chunk in train:
    agg = chunk.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)

aggs = pd.concat(aggs, axis=0)

--------------------------------------



,srch_destination_id,hotel_cluster,sum,count
0,1,20,0,2
1,1,30,0,1
2,1,60,0,2
3,4,22,1,2
4,4,25,1,2


In [6]:
CLICK_WEIGHT = 0.05
agg = aggs.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,0,3,0,2,0.10
1,1,20,4,22,5.10
2,1,30,2,20,3.00
3,1,57,0,1,0.05
4,1,60,0,17,0.85


In [7]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] 

In [8]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
srch_destination_id,
0,3
1,20 30 60 57
2,20 30 53 46 41
3,53 60
4,82 25 32 58 78


In [9]:
test = pd.read_csv(os.path.join(data_dir, 'test.csv'),
                    dtype={'srch_destination_id':np.int32},
                    usecols=['srch_destination_id'],)

In [10]:
test = test.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)
test.head()

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5
2,11353,0 31 77 91 96
3,8250,1 45 79 24 54
4,11812,91 42 2 48 59


In [11]:
most_pop_all = agg.groupby('hotel_cluster')['relevance'].sum().nlargest(5).index
most_pop_all = np.array_str(most_pop_all)[1:-1]
most_pop_all

'91 48 42 59 28'

In [12]:
test.hotel_cluster.fillna(most_pop_all, inplace=True)

In [31]:
test.head()

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5
2,11353,0 31 77 91 96
3,8250,1 45 79 24 54
4,11812,91 42 2 48 59


In [38]:
pop_set = set(['91', '48', '42', '59', '28'])
def add(x):
    cl = x['hotel_cluster']
    if isinstance(cl, int):
        ids_list = [str(cl)]
    else:
        try:
            ids_list = [str(i) for i in x['hotel_cluster'].split(' ')]
        except ValueError:
            ids_list = list(pop_set)
    
    if len(ids_list) < 5:
        free = pop_set - set(ids_list)
        for i in free:
            ids_list.append(i)
            if len(ids_list) >= 5:
                break
        x['hotel_cluster'] = ' '.join(ids_list)
    return x

In [39]:
new_test = test.apply(add, axis=1)

In [41]:
new_test.hotel_cluster.to_csv('simple_stat_2.csv',header=True, index_label='id')

In [20]:
type(set([1, 2])

set